In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
import re
from pandas.io.json import json_normalize 
import json

In [ ]:
kick=pd.read_csv("Kickstarter_projects_Feb19.csv")
kick.head()

In [ ]:
print(kick.columns)
kick.rename(columns={'launched_at': 'inicio_camp', 'deadline': 'fin_camp','goal_usd': 'meta','usd_pledged': 'dinero_reco','currency': 'moneda'}, inplace=True)
print(kick.shape)
print(kick.dtypes)


In [ ]:
print(kick.status.value_counts())
kick.dinero_reco.value_counts()

In [ ]:
#Lista donde se almacenara todo el contenido requerido del scrapeo
dfidea=[]

In [ ]:
#Scrap de Ideame
ideamee=[]
url = 'https://www.idea.me/projects?categories=%s'
pag = 3
see_p = 4

class Scrappag:

    def __init__(self, url_pattern, pages_to_scrape=1, see_more=1, content_parser=None):
        self.url_pattern = url_pattern
        self.pages_to_scrape = pages_to_scrape
        self.content_parser = content_parser
        self.see_more = see_more
         
    def scrape_url(self, url):
        navegador = webdriver.Chrome()
        navegador.get(url)
        for u in range(1, self.see_more+1):
            navegador.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            see_more = navegador.find_element_by_xpath('/html/body/div[4]/section[4]/div/div/a').click()
            time.sleep(3)
        pagina_completa = navegador.page_source
        response = pagina_completa
        time.sleep(1)
        result = self.content_parser(response)
        self.output_results(result)
        navegador.close()
        
    def output_results(self, r):
        dfidea.append(r)
        return
          
    def kickstart(self):
        lista=[1,2,4,5,6,8,9,10,13]
        for i in lista:
            print("absorviendo pag: ",i)
            self.scrape_url(self.url_pattern % i)
    
def quotes_parser(content):
    soup = bs(content, "html.parser")
    art=soup.find_all('article',{'class':'itemProject standardProject whiteProject'})
    
    titles=[]
    recau=[]
    dias=[]
    
    for u in range(len(art)):
        boxtitle=art[u].find_all('div',{'class':'mod-1col title'})
        title=boxtitle[0].find("h4")
        goal=art[u].find_all('div',{'class':'goal'})
        money=goal[0].find('span',{'class':'value'} )
        goal=art[u].find_all('div',{'class':'goal'})
        por=goal[0].find('ul',{'class':'mod-1col stats'} )
        li=por.find("li")
        p=li.text
        k=re.findall('\d+',p)
        l=money.text
        money
        x=title.text
        titles.append(x)
        recau.append(l)
        dias.append(k)
        
    tour={'titulos':titles, 'dias':dias,
                'recau':recau}
    return tour
    
ideame = Scrappag(url,pag,see_p, content_parser=quotes_parser)   
        

In [ ]:
#Disparador del scrap
ideame.kickstart()

In [ ]:
#Agregando columna de category
ArtesV=pd.DataFrame(dfidea[0])
ArtesV["main_category"]="Artes Visuales"
Diseño=pd.DataFrame(dfidea[1])
Diseño["main_category"]="Diseño"
Cine=pd.DataFrame(dfidea[2])
Cine["main_category"]="Cine y Video"
ArtesE=pd.DataFrame(dfidea[3])
ArtesE["main_category"]="Artes Escenicas"
Musica=pd.DataFrame(dfidea[4])
Musica["main_category"]="Musica"
Editorial=pd.DataFrame(dfidea[5])
Editorial["main_category"]="Editorial"
ImpactoS=pd.DataFrame(dfidea[6])
ImpactoS["main_category"]="Impacto Social"
Educacion=pd.DataFrame(dfidea[7])
Educacion["main_category"]="Educacion"
Empre=pd.DataFrame(dfidea[8])
Empre["main_category"]="Emprendimiento"

In [ ]:
dff=ArtesV.append ([Diseño,Cine,ArtesE,Musica,Editorial,ImpactoS,Educacion,Empre], ignore_index=True)
dff.rename(columns={'recau': 'usd_pledged','titulos': 'name'}, inplace=True)
dff.head()

In [ ]:
kick2=kick.append(dff,ignore_index=True)

In [ ]:
#pruebas argentina, no pude aplicar
url = 'https://www.idea.me/projects?categories=13'
html=requests.get(url).content
soup = bs(html, "html.parser")
art=soup.find_all('article',{'class':'itemProject standardProject whiteProject'})
porce=[]
for w in range(len(art)):
    goal=art[w].find_all('div',{'class':'goal'})
    por=goal[0].find('div',{'class':'percent transparent'} )
    p=por.text
    k=re.findall('\d+%',p)
    porce.append(k)
porce 

In [5]:
#Api Kickstart, la pagina no tenia api oficial por lo que hice request directamente en su URL y maneje los JSON

url = "https://www.kickstarter.com/projects/search.json?search=&term=%s"

class jsonn:

    def __init__(self, url_pattern, content_parser=None):
        self.url_pattern = url_pattern
        self.content_parser = content_parser
         
    def scrape_url(self, url):
        res=requests.get(url)
        response = res.json()
        time.sleep(2)
        result = self.content_parser(response)
        self.output_results(result)
        
    def output_results(self, r):
        dfArt=r[["name",'country','category.name',"goal","state","currency","pledged"]]
        archivo= "Kick.csv"
        txt=dfArt.to_csv(index=False)
        csv=open(archivo,"a")
        csv.write(txt)
        
        return 
          
    def kickstart(self):
        lista=["Arte","Comic","Crafts","Dance","Technology","Games","Food","Film","Fashion"]
        for i in lista:
            print("Get category: ",i)
            self.scrape_url(self.url_pattern % i)
    
def quotes_parser(content):
    df1=json_normalize(content["projects"])
    
    return df1
    
kick = jsonn(url, content_parser=quotes_parser)  

In [3]:
#Disparador Api
kick.kickstart()

Get category:  Arte
Get category:  Comic
Get category:  Crafts
Get category:  Dance
Get category:  Technology
Get category:  Games
Get category:  Food
Get category:  Film
Get category:  Fashion


In [3]:
kii=pd.read_csv("Kick.csv")
kii['category.name'].value_counts()
kickfinal=kick2.append(kii)
kickfinal.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 6: invalid continuation byte